<a href="https://colab.research.google.com/github/DmeansDream/AI_Lab_Work/blob/main/AI_Lab_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic imports of libraries and dataset from Google Drive with unzipping and extraction of training data**

In [ ]:
!pip install transformers -q

import os
import zipfile
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive mounted")


In [ ]:
zip_path = "/content/drive/MyDrive/EuroSAT/EuroSAT_RGB.zip"

if os.path.exists(zip_path):
    print("Found dataset")
else:
    print("File not found")

In [ ]:
extract_path = "./data"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzipped successfully.")

    base_dir = os.path.join(extract_path, "2750")
    if not os.path.exists(base_dir):
        possible_dirs = [d for d in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, d))]
        if possible_dirs:
            base_dir = os.path.join(extract_path, possible_dirs[0])

    print(f"Data directory at: {base_dir}")
else:
    print(f"ERROR: Could not find file")

**Preparation of data for machine learning algorithms, involves converting data to tensors, splitting the dataset on training and testing and falttening the images into a vector**

In [ ]:
# Convert to tensor
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(root=base_dir, transform=transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

gen = torch.Generator().manual_seed(393)
train_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset,
    [train_size, test_size],
    generator = gen)

print(f"Total images: {len(full_dataset)}")
print(f"Training set: {len(train_dataset)}")
print(f"Test set: {len(test_dataset)}")
print(f"Classes: {full_dataset.classes}")


# Data flattening function
def extract_numpy_data(dataset):
    loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    data_iter = iter(loader)
    images, labels = next(data_iter)

    X = images.view(images.size(0), -1).numpy()
    y = labels.numpy()
    return X, y

print("Flattening data")
X_train, y_train = extract_numpy_data(train_dataset)
X_test, y_test = extract_numpy_data(test_dataset)

print(f"Data ready: {X_train.shape}")

**Helper training and basic evaluation function, and declaration of training subsets, that were reduced to 5000, to improve training time.**

In [ ]:
import time
from sklearn.metrics import accuracy_score, classification_report

SUBSET_SIZE = 5000

X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train,
    y_train,
    train_size=SUBSET_SIZE,
    stratify=y_train,
    random_state=42
)

def train_and_evaluate(model, model_name):
    print(f"\n{model_name} Training")
    start_time = time.time()

    model.fit(X_train_sub, y_train_sub)

    train_time = time.time() - start_time
    print(f"Training Time: {train_time:.2f} seconds")

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {acc*100:.2f}%")

    return train_time, acc

**First algorithm to try learning the dataset will be Naive Bayes, it is the most basic classification algorithm, here we use default setting for Gaussian Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_time, nb_acc = train_and_evaluate(nb_model, "Naive Bayes")

**Second algorithms is Logistic Regression with semi standard hyper parameters.**

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    solver='lbfgs',
    max_iter=500,
    n_jobs=-1
)

log_time, log_acc = train_and_evaluate(log_reg, "Logistic Regression")

**Now we start training more sophisticated classifier algorithms, starting with Random Forest, that will see a good amount of improvement in comparison to previous models thanks to it's decision-tree structure and ability to recognize basic patterns in dataset**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=393
)

rf_time, rf_acc = train_and_evaluate(rf_model, "Random Forest")

**Now we set up the SVM, where we should see drastic improvement in comparison to Naive Bayes and Log. Reg., since SVM can decipher not only colors but also more complex relationships between them. The difference with RF will be minimal.**

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(
    kernel='rbf',
    C=1.0,
    cache_size=1000
)

svm_time, svm_acc = train_and_evaluate(svm_model, "SVM")

**Then we set up data for uploaded pre trained transformer model.**

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

vit_processor = ViTImageProcessor.from_pretrained(
    "google/vit-base-patch16-224-in21k"
)

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=vit_processor.image_mean,
        std=vit_processor.image_std
    )
])

# Import dataset again using transform parameters
vit_dataset = datasets.ImageFolder(
    root=base_dir,
    transform=vit_transform
)

# Train/test split
train_size = int(0.8 * len(vit_dataset))
test_size = len(vit_dataset) - train_size

train_dataset_vit, test_dataset_vit = torch.utils.data.random_split(
    vit_dataset,
    [train_size, test_size],
    generator=torch.Generator().manual_seed(393)
)

train_loader = DataLoader(train_dataset_vit, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_vit, batch_size=32, shuffle=False)

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=10
)

model.to(device)
print("Model ready\n")

**Conduct training and evaluation of transformer model.**

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

print("ViT Training")
start_time = time.time()
model.train()

for batch in tqdm(train_loader):
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    outputs = model(images, labels=labels)
    loss = outputs.loss

    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

vit_train_time = time.time() - start_time
print(f"Training Time: {vit_train_time:.2f} seconds")

print("Evaluating")
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(test_loader):
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

vit_acc = correct / total
print(f"\n Transformer Accuracy: {vit_acc*100:.2f}%")

**Snippet to save trained models for later reuse**

In [ ]:
from google.colab import drive
import joblib
import os

save_path = "/content/drive/MyDrive/EuroSAT_Models/models"
trans_path = "/content/drive/MyDrive/EuroSAT_Models/models/transformer"

if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save ViT
print(f"Saving model to {trans_path}...")
model.save_pretrained(trans_path)
print("Saved ViT.")

# Save Naive Bayes
joblib.dump(nb_model, os.path.join(save_path, 'nb_model.pkl'))
print("Saved Naive Bayes Regression.")

# Save Logistic Regression
joblib.dump(log_reg, os.path.join(save_path, 'log_reg_model.pkl'))
print("Saved Logistic Regression.")

# Save SVM
joblib.dump(svm_model, os.path.join(save_path, 'svm_model.pkl'))
print("Saved SVM.")

# Save Random Forest
joblib.dump(rf_model, os.path.join(save_path, 'rf_model.pkl'))
print("Saved Random Forest.")

print("Models saved.")

**Snippet for loading trained models**

In [ ]:
import joblib
from transformers import ViTForImageClassification


path = "/content/drive/MyDrive/EuroSAT_Models/models"

nb_model = joblib.load(f"{path}/nb_model.pkl")
log_reg = joblib.load(f"{path}/log_reg_model.pkl")
svm_model = joblib.load(f"{path}/svm_model.pkl")
rf_model = joblib.load(f"{path}/rf_model.pkl")

model = ViTForImageClassification.from_pretrained(f"{path}/transformer")
model.to(device)

print("Models loaded")

**This code snippet will pick a random image from the original test set and feed it to each of the trained models in their respective formats. Then we will be able to see guesses by each model.**

In [ ]:
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image

idx = random.randint(0, len(test_dataset) - 1)
image_tensor, label_idx = test_dataset[idx]
true_label = full_dataset.classes[label_idx]

# Display
plt.figure(figsize=(3, 3))
plt.imshow(image_tensor.permute(1, 2, 0))
plt.title(f"True Label: {true_label}", color='green', fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()

print(f"\n Model Predictions for Image {idx}")

# Prepare data
img_flat = image_tensor.view(1, -1).numpy()

classes = full_dataset.classes

# Naive Bayes
pred_nb = nb_model.predict(img_flat)[0]
print(f"Naive Bayes: {classes[pred_nb]}")

# Logistic Regression
pred_lr = log_reg.predict(img_flat)[0]
print(f"Logistic Regression: {classes[pred_lr]}")

# Random Forest
pred_rf = rf_model.predict(img_flat)[0]
print(f"Random Forest: {classes[pred_rf]}")

# SVM
pred_svm = svm_model.predict(img_flat)[0]
print(f"SVM (RBF): {classes[pred_svm]}")

# Data for transformer
img_vit = vit_transform(to_pil_image(image_tensor)).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    outputs = model(img_vit)
    probs = F.softmax(outputs.logits, dim=-1)
    confidence, pred_vit_idx = torch.max(probs, dim=-1)
    pred_vit = classes[pred_vit_idx.item()]

print(f"Vision Transformer: {pred_vit} (Confidence: {confidence.item()*100:.1f}%)")

**This code snippet is conducting evaluation of every trained model and creates a pandas table with all main metrics for model comparison**

In [ ]:
import time
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

results = []

# Func for non transformer evaluation
def evaluate(name, model, X, y):
    print(f"Evaluating {name}")

    #Inference Time
    start_time = time.time()
    y_pred = model.predict(X)
    end_time = time.time()

    inference_time = end_time - start_time
    ips = len(X) / inference_time

    # Calculate Metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y, y_pred, average='weighted')
    acc = accuracy_score(y, y_pred)

    return {
        "Model": name,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Inference Speed": int(ips),
        "Predictions": y_pred,
        "True Labels": y
    }

# Evaluate ML
results.append(evaluate("Naive Bayes", nb_model, X_test, y_test))
results.append(evaluate("Logistic Regression", log_reg, X_test, y_test))
results.append(evaluate("SVM (RBF)", svm_model, X_test, y_test))
results.append(evaluate("Random Forest", rf_model, X_test, y_test))

# Evaluate ViT
print("Evaluating ViT")

model.eval()
y_true_vit = []
y_pred_vit = []
start_time = time.time()

with torch.no_grad():
    for batch in test_loader:
        images, labels = batch
        images = images.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs.logits, dim=-1)

        y_true_vit.extend(labels.cpu().numpy())
        y_pred_vit.extend(preds.cpu().numpy())

end_time = time.time()
inference_time = end_time - start_time
ips_vit = len(y_true_vit) / inference_time

# Calculate Metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_true_vit, y_pred_vit, average='weighted')
acc = accuracy_score(y_true_vit, y_pred_vit)

results.append({
    "Model": "Vision Transformer",
    "Accuracy": acc,
    "Precision": precision,
    "Recall": recall,
    "F1-Score": f1,
    "Inference Speed": int(ips_vit),
    "Predictions": np.array(y_pred_vit),
    "True Labels": np.array(y_true_vit)
})

# Create table
df_results = pd.DataFrame(results)
df_display = df_results.drop(columns=["Predictions", "True Labels"]).copy()

cols = ["Accuracy", "Precision", "Recall", "F1-Score"]
df_display[cols] = (df_display[cols] * 100).round(2)

print("\n Comparison table")
display(df_display)

**This snippet is creating a confusion matrix for each model using matplotlib library**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Function to plot a single confusion matrix
def plot_confusion(ax, y_true, y_pred, class_names, title):
    cm = confusion_matrix(y_true, y_pred)

    sns.heatmap(
        cm,
        cmap="Greens",
        ax=ax,
        xticklabels=class_names,
        yticklabels=class_names,
        cbar=False
    )

    ax.set_title(title, fontsize=14, fontweight="bold")
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

class_names = full_dataset.classes

for i, res in enumerate(results):
    plot_confusion(
        axes[i],
        res["True Labels"],
        res["Predictions"],
        class_names,
        f"{res['Model']}\nAcc: {res['Accuracy']*100:.1f}%"
    )

# Hide unused subplots
for j in range(len(results), len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()
